In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("CSAT_Cleaned.xlsx", sheet_name="CSAT answers") 

In [ ]:
theme_dict = {
    "Access_Wait": [
        "wait ", "waiting", "on hold", "hold for", "queue", "in a queue",
        "long time", "long wait", "no one answered", "no one picks up",
        "couldn't get through", "cant get through",
        "line was busy", "lines are busy", "call dropped", "cut off",
        "disconnected", "waited" 
    ],
    
    "Multiple_Contacts": [
        "called again", "call again", "called back", "call back", "ring back",
        "phoned again", "several times", "many times", "multiple times",
        "more than once", "over and over", "keep calling", "kept calling",
        "chasing", "had to call back", "had to ring back", "taking too long", "took too long"
    ],

    "Staff_Positive": [
        "helpful", "very helpful", "so helpful", "polite", "very polite",
        "friendly", "professional", "kind", "patient", "respectful", "lovely",
        "excellent service", "great service", "brilliant", "amazing",
        "above and beyond", "listened", "explained everything",
        "explained clearly", "clear explanation" ,
    ],

    "Staff_Negative": [
        "rude", "unhelpful", "not helpful", "couldn't help",
        "wouldn't help", "didn't care", "didn't listen", "ignored", "hung up",
        "put the phone down", "bad attitude", "poor attitude",
        "unprofessional", "useless", "appalling", "disgusting",
        "unacceptable" ,"hung up on me", "poor service", "ignored me", "bad attitude" ,"wasn't listening"
    ],

    "Comms_Updates": [
        "no update", "no updates", "wasn't updated", "was not updated",
        "wasn't informed", "no communication", "lack of communication",
        "nobody told", "no one told", "nobody called back",
        "no one called back", "didn't call back", "promised to call back",
        "never called back", "waiting for a call back", "waiting for callback"

    ],

    "Resolution_Fix": [
        "not resolved", "unresolved", "still not resolved", "still not sorted",
        "issue remains", "problem remains", "problem came back",
        "came back again", "not fixed", "never fixed", "still leaking","repaired",
        "fixed","no one turned up", "didn't turn up","temporary fix"
    ],

    "Billing_Meter": [
        "bill", "billing", "high bill", "large bill", "bigger bill",
        "overcharged", "over charge", "charge", "charges", "credit",
        "in credit", "debit", "payment", "payment plan", "direct debit",
        "tariff", "rate", "meter", "meter reading", "reading",
        "usage", "consumption", "smart meter"
    ],

    "Digital_Online": [
        "online", "website", "web site", "app", "mobile app", "logged in",
        "login", "account", "my account", "form", "online form",
        "application form", "webchat", "live chat",
        "chatbot", "bot"
    ]
}


In [4]:
# Clean comments: lowercase + fill blanks
df["comment_clean"] = (
    df["comments_verb"]
    .astype(str)
    .str.lower()
    .fillna("")
)


In [5]:
def tag_themes(text, theme_dict):
    """Return a dictionary of theme flags for one text row."""
    flags = {}
    for theme, keywords in theme_dict.items():
        flags[theme] = int(any(kw in text for kw in keywords))
    return flags

# Expand tags into columns
tagged = df["comment_clean"].apply(lambda x: pd.Series(tag_themes(x, theme_dict)))

df = pd.concat([df, tagged], axis=1)

In [6]:
df.head()

,csat_id,Survey_Date,Journey,Sub_Area,Area,Department,Business_Unit,Channel,Score,cid,...,comment_lower,comment_clean,Access_Wait,Multiple_Contacts,Staff_Positive,Staff_Negative,Comms_Updates,Resolution_Fix,Billing_Meter,Digital_Online
0,1,2025-05-01,Invoice/statement - query,Retail,NaN,NaN,Revenue,T,0,10,...,i feel i was bullied into accepting your price...,i feel i was bullied into accepting your price...,0,0,0,0,0,0,1,1
1,2,2025-05-02,Payment scheme/instalment plan - set up,Retail,Thames Water,Thames Water - Case Management,Revenue,T,10,10,...,NaN,nan,0,0,0,0,0,0,0,0
2,3,2025-05-03,Payment Made,Retail,Tech Mahindra,Tech Mahindra - First Contact,Revenue,T,10,10,...,excellent help.,excellent help.,0,0,0,0,0,0,0,0
3,4,2025-05-03,Request a meter - query,Retail,Thames Water,Thames Water - First Contact,Revenue,T,10,10,...,helpfulness of lady i spoke to. she listened t...,helpfulness of lady i spoke to. she listened t...,0,0,1,0,0,0,0,0
4,5,2025-05-04,High Bill -Suspected Leak,Retail,Thames Water,Thames Water - First Contact,Revenue,T,10,10,...,your adviser was so helpful and understanding....,your adviser was so helpful and understanding....,0,0,1,0,0,0,0,0


In [7]:
summary = (
    df.groupby("cid")[list(theme_dict.keys())]
    .mean()
    .reset_index()
)

In [8]:
summary

,cid,Access_Wait,Multiple_Contacts,Staff_Positive,Staff_Negative,Comms_Updates,Resolution_Fix,Billing_Meter,Digital_Online
0,10,0.031739,0.014729,0.335441,0.023853,0.001955,0.011992,0.112357,0.099127
1,42,0.016750,0.008375,0.103853,0.015075,0.000000,0.021776,0.031826,0.067002
2,43,0.029248,0.016713,0.097493,0.030641,0.002786,0.018106,0.178273,0.126741
3,44,0.036806,0.014352,0.197454,0.025926,0.004167,0.029630,0.111574,0.115278


In [9]:
negative = df[df["Sentiment_Score"] < 2]

neg_summary = (
    negative.groupby("cid")[list(theme_dict.keys())]
    .mean()
    .reset_index()
)
neg_summary

,cid,Access_Wait,Multiple_Contacts,Staff_Positive,Staff_Negative,Comms_Updates,Resolution_Fix,Billing_Meter,Digital_Online
0,10,0.111452,0.067573,0.097411,0.141290,0.008776,0.055726,0.332163,0.216323
1,42,0.057692,0.000000,0.076923,0.134615,0.000000,0.115385,0.173077,0.230769
2,43,0.057471,0.063218,0.074713,0.120690,0.005747,0.040230,0.402299,0.264368
3,44,0.103448,0.048751,0.059453,0.118906,0.013080,0.086801,0.297265,0.242568


In [10]:
df.to_excel("CSAT_Cleaned_Tagged.xlsx",sheet_name="CSAT answers", index=False)

In [11]:
theme_cols = list(theme_dict.keys())

df["No_Theme"] = (df[theme_cols].sum(axis=1) == 0).astype(int)


In [12]:
df["No_Theme"].value_counts()

No_Theme
1    11549
0     9430
Name: count, dtype: int64